In [329]:
import pandas as pd
from sqlalchemy import create_engine

In [330]:
import os
print(os.getcwd())

/Users/Victoria/bootcamp/CU-NYC-DATA-PT-10-2019-U-C/project2_ETL


In [331]:
alcohol_csv_file = "resources/drinks.csv"

In [332]:
alcohol_df=pd.read_csv(alcohol_csv_file)
alcohol_df.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
0,Afghanistan,0,0,0,0.0
1,Albania,89,132,54,4.9
2,Algeria,25,0,14,0.7
3,Andorra,245,138,312,12.4
4,Angola,217,57,45,5.9


In [333]:
happiness_csv_file= "resources/2015.csv"

In [334]:
happiness_df=pd.read_csv(happiness_csv_file)
happiness_df.head()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


In [347]:
alcohol_cols = ["country", "total_litres_of_pure_alcohol"]
alcohol_transformed= alcohol_df[alcohol_cols].copy()

# Rename the column headers
alcohol_transformed = alcohol_transformed.rename(columns={"total_litres_of_pure_alcohol": "total_alcohol_consumption",
                                                            })

In [348]:
alcohol_transformed.head()

,country,total_alcohol_consumption
0,Afghanistan,0.0
1,Albania,4.9
2,Algeria,0.7
3,Andorra,12.4
4,Angola,5.9


In [374]:
alcohol_transformed["country"] = alcohol_transformed["country"].map(lambda x: "United States" if x == "USA" else x)

In [349]:
happiness_cols = ["Country", "Happiness Rank", "Happiness Score"]
happiness_transformed= happiness_df[happiness_cols].copy()

# Rename the column headers
happiness_transformed = happiness_transformed.rename(columns={"Country": "country",
                                                              "Happiness Rank": "happiness_rank",
                                                              "Happiness Score": "happiness_score"
                                                            })

In [350]:
happiness_transformed.head()

,country,happiness_rank,happiness_score
0,Switzerland,1,7.587
1,Iceland,2,7.561
2,Denmark,3,7.527
3,Norway,4,7.522
4,Canada,5,7.427


In [351]:
alcohol_happiness_df= pd.merge(alcohol_transformed,happiness_transformed ,on='country', how='inner')

In [352]:
alcohol_happiness_df.sort_values(by=['happiness_rank'], inplace=True)

In [353]:
alcohol_happiness_df.head()

,country,total_alcohol_consumption,happiness_rank,happiness_score
124,Switzerland,10.2,1,7.587
54,Iceland,6.6,2,7.561
34,Denmark,10.4,3,7.527
96,Norway,6.7,4,7.522
23,Canada,8.2,5,7.427


In [354]:
alcohol_happiness_df.set_index(['country'], inplace=True)

In [355]:
alcohol_happiness_df.head()

,total_alcohol_consumption,happiness_rank,happiness_score
country,,,
Switzerland,10.2,1,7.587
Iceland,6.6,2,7.561
Denmark,10.4,3,7.527
Norway,6.7,4,7.522
Canada,8.2,5,7.427


In [359]:
connection_string = "postgres:postgres@localhost:5432/happiness_alcohol_db"
engine = create_engine(f'postgresql://{connection_string}')

In [360]:
engine.table_names()

['happiness_alcohol_consumption']

In [361]:
alcohol_happiness_df.to_sql(name='happiness_alcohol_consumption', con=engine, if_exists='append', index=True)